# Trading Strategy for Finance using LSTMs

### Notebook Configurations and Packages

Let's execute the cell below to display information about the GPUs running on the server. First, we import several widely used modules such as NumPy for numerical calculations, pandas for data management, matplotlib for visualizations, and TensorFlow for building and training deep neural networks.

**Environment Verification**

In [ ]:
!nvidia-smi -l 1

In [37]:
#imports
import h5py
import pandas as pd 
import numpy as np
import pprint as pp 
import tensorflow as tf 
from tensorflow.contrib import rnn
import math
import sklearn
import matplotlib.pyplot as plt
import warnings
from tradingcore import prepareData as prepData
from numpy.random import seed

In [38]:
# setting up random seed
seed(42)
tf.set_random_seed(42)
MAX_SEQUENCE_LENGTH = 32
EMBEDDING_DIM = 300

In [36]:
# print versions
print("Tensorflow version:",tf.__version__)
print("Sklearn version:",sklearn.__version__)

Tensorflow version: 1.13.1
Sklearn version: 0.20.1


#### Data Preparation

A typical DL workflow starts with data preparation because the data is not clean and ready to use most of the time. Deep neural network building and training follow the data preparation. Lastly, the trained network is validated with a dataset. 

The original data needs to be cleaned before training the network. Since cleaning the data takes significant amount of time (around 20 minutes), we have stored the cleaned data into another .h5 file. If you would like to use the original data and run the cleaning code, please set the "usePreparedData" variable to "False".

In [33]:
# The data is prepared and stored in a seperate .h5 file.
# Set usePreparedData = False to use the original data and run the data preparation code
usePreparedData = True
# insampleCutoffTimestamp variable is used to split the data in time into two pieces to create training and test set.
insampleCutoffTimestamp = 1650

# If usePreparatedData is True, then the prepared data is stored. Otherwise, the original data is stored
if usePreparedData == True:
    #with pd.HDFStore("/home/mimas/2sigma/DLI_FSI/2sigma/train_prepared.h5", 'r') as train:
    with pd.HDFStore("../data/algo_training/trainDataPrepared.h5", 'r') as train:
        df = train.get("train") 
else:
    with pd.HDFStore("../data/algo_training/train.h5", 'r') as train:
        df = train.get("train")

There are multiple instruments in the dataset and each instrument has an id. Time is represented by the 'timestamp' feature. Let's look at the data.

In [34]:
# This will print the dataset
df

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_43,technical_44,y,y_lagged,technical_diff,krnldiff,delta5diff,krnl40,delta540,fmod29
0,10,0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.335633,0.113292,1.621238,...,-2.000000e+00,0.000951,-0.011753,0.000046,0.000000,0.000000,0.000000,-0.041838,0.000011,0.666596
1,11,0,0.014765,-0.038064,-0.017425,0.320652,-0.034134,0.004413,0.114285,-0.210185,...,-2.000000e+00,0.000951,-0.001240,0.000046,0.000000,0.000000,0.000000,-0.041838,0.000011,0.666596
2,12,0,-0.010622,-0.050577,1.571245,-0.157525,-0.068550,-0.155937,1.060683,-0.764516,...,-2.000000e+00,0.000951,-0.020940,0.000046,0.006942,0.000000,0.000000,-0.041838,0.000011,0.666596
3,25,0,-0.003429,-0.012705,-0.005859,-0.037375,0.024913,0.178495,0.044287,-0.007262,...,-2.000000e+00,0.000951,-0.015959,0.000046,0.006766,0.000000,0.000000,-0.041838,0.000011,0.666596
4,26,0,0.176693,-0.025284,-0.057680,0.015100,0.180894,0.139445,-0.125687,-0.018707,...,0.000000e+00,0.000951,-0.007338,0.000046,0.006236,0.000000,0.000000,-0.041838,0.000011,0.666596
5,27,0,0.346856,0.166239,-1.482727,-0.992249,-0.125916,0.345812,0.044287,-0.584239,...,-2.000000e+00,0.000951,0.031425,0.000046,0.010000,0.000000,0.000000,-0.041838,0.000011,0.666596
6,31,0,0.072036,0.014931,-0.005859,0.014063,0.024913,-0.193205,0.044287,0.019531,...,-2.000000e+00,0.000951,-0.032895,0.000046,0.006601,0.000000,0.000000,-0.041838,0.000011,0.666596
7,38,0,0.300062,0.071251,-0.074451,-0.065292,-0.011286,0.026365,0.210249,0.167494,...,-2.000000e+00,0.000951,0.015803,0.000046,0.007909,0.000000,0.000000,-0.041838,0.000011,0.666596
8,39,0,-0.003511,-0.034270,0.082372,-0.023937,-0.025750,0.007815,0.263451,-0.241212,...,-2.000000e+00,0.000951,-0.027593,0.000046,0.000000,0.000000,0.000000,-0.041838,0.000011,0.666596
9,40,0,-0.083330,0.081935,-1.482727,-0.206856,-0.839563,-0.234100,-0.291853,-2.522340,...,-2.000000e+00,0.000951,0.006662,0.000046,0.000000,0.000000,0.000000,-0.041838,0.000011,0.666596


If the original data is stored, the data preparation code will be executed in the following cell. First, extreme values in each feature set are removed. Then, some hand-crafted features are added to feature set to boost the prediction accuracy. There are many methods including PCA and auto-encoders to do the feature engineering rather than creating hand-crafted features. As an exercise, we highly recommend you to add auto-encoders to the code and check the accuracy after the lab. Lastly, NaNs are replaced with the median of the feature.

In [35]:
if usePreparedData == False:
    # Original data is not clean and some the samples are a bit extreme.
    # These values are removed from the feature set.
    df = prepData.removeExtremeValues(df, insampleCutoffTimestamp)
    # A little bit feature engineering. Hand-crafted features are created here to boost the accuracy.
    df = prepData.createNewFeatures(df) 
    # Check whether ve still have any NaNs 
    df = prepData.fillNaNs(df) 
    df.to_hdf("data/algo_trading/trainDataPrepared.h5", 'train')


# Trading Strategy for Finance using LSTMs

#### Model Construction

Now, we set up the TensorFlow compute graph. The deep neural network that is used in this code is comprised of a LSTM cell that runs over 10 time steps, a fully connected layers (FCL), and also drop-out layers to prevent overfitting. Calculating the number of time steps for a recurrent neural network is not a trivial task. It is actually another hyperparameter that needs to be searched. The network is depicted in the following figure.

<img src="../imgs/dnn.jpg" width="700" height="700">

        Figure 3: Structure of the LSTM based deep neural network
                                
Below is the code to build the deep neural network depicted in Figure 3;

In [ ]:
def weight_variable(shape): 
    initial = tf.truncated_normal(shape, stddev=0.3)
    return tf.Variable(initial) 
    
def bias_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.3)
    return tf.Variable(initial) 

n_time_steps = 10
def getDNN (x, LSTMCellSize, keep_prob):
    with tf.name_scope('model'):
        with tf.name_scope('RNN'):
            # We will add two dropout layers and LSTM cells with the number of units as LSTMCellSize.
            cell = rnn.DropoutWrapper(rnn.BasicLSTMCell(LSTMCellSize, forget_bias=2, activation=tf.nn.tanh), output_keep_prob=keep_prob)
            # We use the cell to create RNN.
            # Note that outputs is not a tensor, it is a list with one element which is numpy array. 
            outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32) 
            outputs_shape = outputs.get_shape().as_list()
                
        # hidden layer with sigmoid activation
        with tf.name_scope('W_fc1'):
            W_fc1 = weight_variable([LSTMCellSize, 1])
        with tf.name_scope('b_fc1'):
            b_fc1 = bias_variable([1])
        with tf.name_scope('pred'):
            pred = tf.matmul(outputs[:,-1,:], W_fc1) + b_fc1

        return pred
    


In [ ]:
# The column names that will be included in the featureset are added into colList.
# colList will be used throughout the lab.
colList=[]                  
for thisColumn in df.columns: 
    if thisColumn not in ('id', 'timestamp', 'y', 'CntNs', 'y_lagged'): 
        colList.append(thisColumn)
colList.append('y_lagged')

#if you do not reset the default graph you will need to restart the kernel
#every time this notebook is run
tf.reset_default_graph()

# Network Parameters 
# Number of units in the LSTM cell.
n_LSTMCell = len(colList)

# Placeholder for the input and the keep probability for the dropout layers
with tf.name_scope('input'):
    x= tf.placeholder(tf.float32, shape=[None, n_time_steps, len(colList)])
with tf.name_scope('keep_prob'):
    keep_prob = tf.placeholder(tf.float32)

# At the input, we create 2-layer LSTM cell (with dropout layers)
print('Building tensorflow graph')

# Graph construction for the LSTM based deep neural network. 
# Structure of the network is depicted in the above figure.
# Please see the dnn.py to see the code of the network.
pred = getDNN (x, n_LSTMCell, keep_prob)

#### Training and Testing

We split the data into two pieces in time to have a training and testing set. In order to have enough sample for each id, the cut-off timestamp for the training set was defined in "insampleCutoffTimestamp" variable as 1650. Figure 4 shows how an instrument is split in time to create training and testing set. While training the model, the training set for each instrument will be fed separately to learn the time patterns in the data.

<img src="imgs/data_split.jpg" width="600" height="600">

                    Figure 4: Training and Testing Dataset
    
In the Kaggle challenge, the metric to evaluate the prediction accuracy was given as Pearson correlation. In statistics, [pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) is a measure of the linear correlation between two variables X and Y. It has a value between +1 and −1, where 1 is total positive linear correlation, 0 is no linear correlation, and −1 is total negative linear correlation. It is widely used in the sciences. It was developed by Karl Pearson from a related idea introduced by Francis Galton in the 1880s.

Depending on the frequency of the financial data, Pearson correlation (R) can be very small. In finance, given the high ratio of signal-to-noise, even a small R can deliver meaningful value. Please note that the algorithm that won the challenge had only 0.038 R.

The following cell includes the code for creating training and testing set, and calculating Pearson correlation.

In [ ]:
# Placeholder for the output (label)
with tf.name_scope('label'):
    y = tf.placeholder(tf.float32, shape=[None, 1]) 
# Placeholder to be able to split the data into training and test set while training the network.
inSampleCutoff = tf.placeholder(tf.int32, shape = ())

# this is important - we only want to train on the in-sample set of rows using TensorFlow
y_inSample = y[0:inSampleCutoff]
pred_inSample = pred[0:inSampleCutoff]

# also extract out of sample predictions and actual values,
# we'll use them for evaluation while training the model.
y_outOfSample = y[inSampleCutoff:]
pred_outOfSample = pred[inSampleCutoff:]

with tf.name_scope('stats'):
    # Pearson correlation to evaluate the model
    covariance = tf.reduce_sum(tf.matmul(tf.transpose(tf.subtract(pred_inSample, tf.reduce_mean(pred_inSample))),tf.subtract(y_inSample, tf.reduce_mean(y_inSample))))
    var_pred = tf.reduce_sum(tf.square(tf.subtract(pred_inSample, tf.reduce_mean(pred_inSample))))
    var_y = tf.reduce_sum(tf.square(tf.subtract(y_inSample, tf.reduce_mean(y_inSample))))
    pearson_corr = covariance / tf.sqrt(var_pred * var_y) 

tf.summary.scalar("pearson_corr", pearson_corr)

In most of the traditional machine learning and deep learning methods, it is assumed that the feature set and predicted value have zero mean and unit variance gaussian distribution. Empirical studies show that the financial data such as asset returns is often not compatible with this assumption. That is why we normalize the "y" variable by subtracting its mean and dividing the result by the standard deviation in the following cell. As an exercise, you can also normalize the features and see if you improve the accuracy.

In [ ]:
# Training dataset is also created here. We included the code to split the data in the above cell. 
# The difference is that the above code will be used in the training by the TensorFlow.
# This code will not be used by TensorFlow and creates the testing dataset whenever it is executed.
dfInSample = df[df.timestamp <  insampleCutoffTimestamp]
# create a reference dataframe (that only depends on in-sample data)
# that gives us standard deviation and mean information on per-id basis
# we'll use it later for variance stabilization
meanStdById = dfInSample.groupby(['id']).agg( {'y':['mean', 'std']})

We are ready to launch the graph for training the model and see intermediate diagnostics results and the final result. We defined the important hyperparameters including the epoch, training batch size and learning rate at the top of the cell. Initially, the epoch is set to 1 because it takes 15-20 minutes to complete the training with 10 epochs even though we are using GPUs. In order to speed up the training in the lab environment, we provided pre-trained networks with 10 epochs and 20 epochs. An adaptive learning rate starting from 0.002 with exponential decay is used for the training from scratch. Learning rate should be set to 0.00058 and 0.00061 for using pre-trained models with 10 and 15 epochs respectively.

In [ ]:
# Training parameters
display_step = 100 
epoch = 1
pre_trained_model = 'tasks/task2/task/SavedModels/model_epoch_10.ckpt'
mini_batch_limit = 1300

# set up adaptive learning rate:
globalStep = tf.placeholder(tf.float32)
# Ratio of globalStep / totalDecaySteps is designed to indicate how far we've progressed in training.
# the ratio is 0 at the beginning of training and is 1 at the end.
# adaptiveLearningRate will thus change from the starting learningRate to learningRate * decay_rate
# in order to simplify the code, we are fixing the total number of decay steps at 1 and pass globalStep
# as a fraction that starts with 0 and tends to 1.
# Learning rate should be set to 0.002 if you are training from scratch.
# Learning rate should be set to 0.00058 if you are using the pre-trained network with 10 epochs.
# Learning rate should be set to 0.00061 if you are using the pre-trained network with 15 epochs.
adaptiveLearningRate = tf.train.exponential_decay(
  0.00058,       # Start with this learning rate
  globalStep,  # globalStep / totalDecaySteps shows how far we've progressed in training
  1,           # totalDecaySteps
  0.3)         # decay_rate, the factor by which the starting learning rate will be 
               # multiplied when the training is finished
    
# Define loss and optimizer
# Note the loss only involves in-sample rows
# Regularization is added in the loss function to avoid over-fitting
rnn_variables = lstm_variables = [v for v in tf.trainable_variables()
                    if v.name.startswith('rnn')]

with tf.name_scope('loss'):
    loss = tf.nn.l2_loss(tf.subtract(y_inSample,pred_inSample)) + tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(scale=0.0001), tf.trainable_variables())

tf.summary.scalar("loss", loss)
optimizer = tf.train.AdamOptimizer(learning_rate=adaptiveLearningRate).minimize (loss) 

# Getting unique ids to train the network per id basis.
ids = df.id.unique()
ids.sort()

summary_op = tf.summary.merge_all()

# initialize the variables 
init = tf.global_variables_initializer()

totalActual = []
totalPredicted = []
import random
# Launch the graph 
# Implement Cross Validation, but in a vay that preserves temporal structure for id's 
with tf.Session() as sess:  
    # Global variables are initialized
    sess.run(init) 
    
    # Restore latest checkpoint
    model_saver = tf.train.Saver()
    model_saver.restore(sess, pre_trained_model)
    
    writer = tf.summary.FileWriter("tasks/tensorboard/logs", graph=tf.get_default_graph())
    step = 50  
    writer_step = 1;
    for i in range(epoch):
        print('Epoch: ', i, '******************************')        
        actual = []
        predicted = []
        
        random.shuffle(ids)

        for thisId in ids:
            # Getting the data of the current id
            this_df = df[df.id == thisId].copy()
            this_df = this_df.sort_values(['id', 'timestamp'])
                        
            # we need to pass training set to the graph definition
            # optimization will only consider in training set
            inSampleSize, _ = this_df[this_df.timestamp < insampleCutoffTimestamp].shape
            totalRows, _ = this_df.shape
            
            batch_y = this_df.loc[:,'y'].values            
            batch_x = this_df[colList].values
                    
            if totalRows < n_time_steps:
                continue

            # Data is formated as a 3D tensor with the shape of (batch_size, n_time, n_feature) for LSTM
            # n_time_steps parameter determines how many steps that LSTM will unroll in time
            complete_x = np.zeros([totalRows-n_time_steps+1, n_time_steps, len(colList)])
            for n in range(n_time_steps):
                complete_x[:,n,:]=batch_x[n:totalRows-n_time_steps+n+1,:]
            
            batch_y = batch_y[n_time_steps-1:]
            inSampleSize -= n_time_steps - 1

            # variance stabilizing transform
            # some id's will not have in-sample rows, we cannot perform transform on those
            # furthermore, since there is not in-sample rows to train on, we must skip
            if inSampleSize < 10:
                continue
                
            # perform variance stabilization
            thisMean = meanStdById.loc[thisId][0]
            thisStd = meanStdById.loc[thisId][1]
            batch_y = (batch_y - thisMean) / thisStd
            
            batch_y = batch_y.reshape(-1,1)
            minibatchSize, _ = batch_y.shape

            # we want to make sure that RNN reaches steady state
            if minibatchSize < mini_batch_limit: 
                continue 
            
            # Run optimization 
            # note: keep_prob is set to 0.5 for training only!
            _, currentRate = sess.run([optimizer, adaptiveLearningRate], feed_dict={x: complete_x, y: batch_y, keep_prob:0.5, inSampleCutoff:inSampleSize, globalStep:i/epoch})

            # Obtain out of sample target variable and our prediction
            y_oos, pred_oos = sess.run([y_outOfSample, pred_outOfSample], feed_dict={x: complete_x, y: batch_y, keep_prob:1.0, inSampleCutoff:inSampleSize}) 
            
            # flatten the returned lists
            y_oos = [y for x in y_oos for y in x]
            pred_oos = [y for x in pred_oos for y in x]
            
            #reverse transform before recording the results
            if inSampleSize:            
                y_oos = [ (t*thisStd + thisMean) for t in y_oos]
                pred_oos = [ (t*thisStd + thisMean) for t in pred_oos]
            
            # record the results
            actual.extend(y_oos)
            predicted.extend(pred_oos)
                       
            totalActual.extend(y_oos)
            totalPredicted.extend(pred_oos)
            
            # Once every display_step show some diagnostics - the loss function, in-sample correlation, etc.
            if step % display_step == 0: 
                # Calculate batch accuracy 
                # Calculate batch loss 
                correl, lossResult, summary = sess.run([pearson_corr, loss, summary_op], feed_dict={x: complete_x, y: batch_y, keep_prob:1.0, inSampleCutoff:inSampleSize})
                
                writer.add_summary(summary, writer_step)
                writer_step += 1
                # corrcoef sometimes fails to compute correlation for a perfectly valid reason (e.g. stdev(pred_oos) is 0)
                # it sets the result to nan, but also gives an annoying warning
                # the following suppresses the warning
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    correl_oos = np.corrcoef(y_oos, pred_oos)[0,1]
                    
                print('LR: %s - Iter %s, minibatch loss = %s, minibatch corr = %s, oos %s (%s/%s)' % (currentRate, step, lossResult, correl, correl_oos, inSampleSize, totalRows))
                
            step += 1 
       
        print('Optimization Finished!') 
        print('Correl: ', np.corrcoef(actual, predicted)[0,1])

It takes 3-5 minutes to complete the training with 1 epochs. We also provided TensorBoard to review the model architecture, loss and correlation variables. TensorBoard is a suite of web applications for inspecting and understanding your TensorFlow runs and graphs. 

### Click [here](/tensorboard/) to start TensorBoard.

You should get a correlation value around R = 0.04. Note that the correlation tends to increase with each epoch (but not always). 

Let's use the pre-trained model with 15 epochs by setting the pre_trained_model variable as **pre_trained_model = '/tasks/task2/task/SavedModels/model_epoch_15.ckpt'** in above cell, lower the starting Learning Rate to **0.00061** and re-run everything using Kernel->Restart & Run All.

What is the correlation that you get this time? Was it improved?

Since training takes significant amount of time, we recommend you train the model with 20 epochs and check the correlation after this lab in your environment. You should get a correlation value around R = 0.05.


# Trading Strategy for Finance using LSTMs

## Optional Exercise
Please read ahead and only come back to these optional exercises if time permits.

**Train from scratch** [20-30 mins]

First, change the # of epochs to 20 in the above cell. Second, put the starting learning rate back to **0.002**. Third, comment out the two line where the pre-trained model is loaded (under "Restore latest checkpoint"). Then re-run everything using Kernel->Restart & Run All. 



#### How can a portfolio manager assess the predicted signal?

We could scatter-plot actual returns over the predicted returns, however correlation is not visually apparent on scatter plots when the correlation is below 20-30%. The correlation we achieve in this signal is much weaker which is typical of modern financial markets. Correlations which we often observe in other applications of predictive models are all but impossible in the financial markets which are highly efficient (simply put, unpredictable). If we imagine that someone has a signal with correlation of 30% using leverage the person would soon get extremely rich - and the observed signal (inefficiency) would disappear from the market.

In order to visually assess the signal, we split out of sample data points into buckets based on the value of predicted returns. We then compute per-bucket mean actual returns. Then we plot mean actual returns (Y axis) against predicted returns (X axis). We thus plot one point per bucket. By taking mean value, we average out the variance within each bucket and uncover the predictive value of the signal.

In [ ]:
actual = totalActual
predicted = totalPredicted

actualMeanReturn = []
predictedMeanReturn = []
stdActualReturns = []
# Buckets are created
buckets = np.arange(-0.02,0.02,0.002)

actual = np.array(actual)
predicted = np.array(predicted)

# Predicted values and the actual values are placed into buckets
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i in range(len(buckets)-1):
        index = np.logical_and(predicted>buckets[i], predicted<buckets[i+1])
        thisBucket = actual[index].mean()
        actualMeanReturn.append(thisBucket)
        predictedMeanReturn.append(predicted[index].mean())
        stdActualReturns.append(actual[index].std())

# Actual versus predicted values are plotted
plt.figure()
plt.plot(predictedMeanReturn,actualMeanReturn, marker='*')
plt.xlabel('predicted')
plt.ylabel('actual')
plt.grid(True)
plt.show()

plt.figure()
plt.errorbar(predictedMeanReturn, actualMeanReturn, yerr = stdActualReturns, marker='*')
plt.xlabel('predicted')
plt.ylabel('actual')
plt.grid(True)
plt.show()


**How much variance is there?**

Plot 2 answers this question by adding error bars to the previous plot. Length of the error bar is equal to the standard deviation of actual returns within each respective bucket.
Plots such as these would be typically used by a portfolio manager to assess behavior of prospective signals and to assess signal levels at which an action should be taken. The simplest trading system utilizing this signal would buy security when predicted return is above some threshold (say, above 0.5%) and sell (or short-sell) the security when the signal is below negative threshold (e.g. below -0.5%). 


## 4. Next Steps

We recommend you to try the following steps after the lab.

1. Try using other machine learning techniques such as random forest, ridge regression, xgboost and compare the correlation with LSTM based predictor.

2. Try using autoencoder to extract fewer features than the original dataset provides and use the features as input to the deep learning model. Analyze the performance. 

## 5. Summary

In this lab, step by step implementation of a LSTM based deep neural network to predict time series financial data is presented. The performance of the model is evaluated with the pearson correlation and competitive performance is achieved. The code provided in this lab can be used in complex trading strategies.

## 6. Post-Lab

Finally, don't forget to save your work from this lab before time runs out and the instance shuts down!!

1. You can download the data from this [https://www.kaggle.com/c/two-sigma-financial-modeling](https://www.kaggle.com/c/two-sigma-financial-modeling).

2. To use the data, please set the "usePreparedData" variable to False before running the code on your environment.

3. Also, remove the code "model_saver.restore(sess, pre_trained_model)" to train the model for you data.


